In [1]:
import digitalhub as dh

project = dh.get_or_create_project("td733-demo-early-exit-eng")

In [4]:
train_fn = project.new_function(name="train-eng-function",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="git+https://github.com/eb-tndigit/audio-early-exit-transformer.git",
                                handler="train_eng_model:dh_train",
                                requirements=["torch==2.5.0", "torchaudio==2.5.0", "tensorboard==2.18.0", "flashlight==0.1.1", "flashlight-text==0.0.7", "sentencepiece==0.2.0",
                                              "soundfile==0.12.1", "editdistance==0.8.1"])

In [5]:
train_fn.run(action="job", 
             resources = { "cpu": { "limits": "4"  },  "mem" : { "limits": "24Gi" }, 
                          "gpu": { "requests": "1", "limits": "1"  }},
             tolerations = [{"key": "nvidia.com/gpu", "operator": "Equal", 
                             "value": "h100", "effect": "NoSchedule"}],           
             parameters={"librispeech_train_dataset": "train-clean-100", 
                         "num_epochs": 10, 
                         "model_name": "early-exit-eng-model", 
                         "base_dir": "/shared/"},
            # HERE: perché sono necessary i volumes?
             volumes=[
                {
                    "volume_type": "persistent_volume_claim", 
                    "name": "early-exit-demo-shared", 
                    "mount_path": "/shared", # HERE: perché lo monta in shared?
                    "spec": { "claim_name": "early-exit-demo-shared",  "size": "20Gi" }
                },
                {
                    "volume_type": "persistent_volume_claim", 
                    "name": "early-exit-demo-data", 
                    "mount_path": "/data", 
                    "spec": { "claim_name": "early-exit-demo-data",  "size": "20Gi" }
                },
            ])
# aggiunto size a 30GB dentro a spec per 0.11
# Questo per shared
# https://www.openslr.org/resources/12/train-clean-100.tar.gz questo sono 6,9GB
# Questo per data
# https://www.openslr.org/resources/12/test-clean.tar.gz 300MB

{'kind': 'python+run', 'metadata': {'project': 'td733-demo-early-exit-eng', 'name': '0600c4d0a0b44a9aababd3962a683744', 'created': '2025-08-14T12:23:46.155Z', 'updated': '2025-08-14T12:23:46.176Z', 'created_by': 'emanuele.boscari@tndigit.it', 'updated_by': 'emanuele.boscari@tndigit.it', 'relationships': [{'type': 'run_of', 'dest': 'store://td733-demo-early-exit-eng/function/python/train-eng-function:c250b59555514a16bb58db066945b725'}]}, 'spec': {'task': 'python+job://td733-demo-early-exit-eng/d1892827d10a4d2380787301ef8884c5', 'local_execution': False, 'function': 'python://td733-demo-early-exit-eng/train-eng-function:c250b59555514a16bb58db066945b725', 'volumes': [{'volume_type': 'persistent_volume_claim', 'name': 'early-exit-demo-shared', 'mount_path': '/shared', 'spec': {'size': '20Gi'}}, {'volume_type': 'persistent_volume_claim', 'name': 'early-exit-demo-data', 'mount_path': '/data', 'spec': {'size': '20Gi'}}], 'resources': {'cpu': {'limits': '4'}, 'mem': {'limits': '24Gi'}, 'gpu': 